# Use NCBI Datasets to Retrieve Information on Genes

The objective of this notebook is to use the `ncbi.datasets` python library to demonstrate how to download and extract gene data from NCBI Datasets.

There are two major types of gene data available from NCBI Datasets:

1. Gene datasets, which include gene, transcript, and protein sequences, and a data report (gene metadata in JSON lines format)

2. Gene summaries, which are brief descriptions of the gene datasets described above

As an example, we will get gene data for the Gonadotropin Releasing Hormone Receptor (GNRHR) gene family, which plays a key role in sexual development and function across vertebrates.

While the role of GnRH in reproduction is conserved across vertebrates, GnRH receptor copy number is variable. 
In humans and some primates, there are two GnRHR genes, while in fish and amphibians, three GnRHR genes have been identified, with additional duplications observed in some fish. Additional receptors and GnRH ligands suggests that new functions could have been acquired by the additional gene copies[1].

We expect to observe this variable gene copy number in the data we obtain using NCBI Datasets.

[1] Moncaut N, Somoza G, Power DM, Canário AV. Five gonadotrophin-releasing hormone receptors in a teleost fish: isolation, tissue distribution and phylogenetic relationships. J Mol Endocrinol. 2005 Jun;34(3):767-79. doi: 10.1677/jme.1.01757. PMID: 15956346.


First we will load the libraries necessary to run this notebook. This includes:
1. the ncbi.datasets library 
2. pandas, the python data analysis library and 
3. pprint, which allows "pretty-printing" of Python data structures

In [1]:
# load all libraries
import pandas as pd
import ncbi.datasets

## Get gene summaries for three human GnRHR genes 
First we're going to get gene summaries for three human GnRHR genes, GNRHR, GNRHR2, and GNRHR2P1, by specifying the NCBI Gene IDs for these genes. 

Gene summaries contain a lot of interesting metadata and it's easy to pull out just the fields that you're interested in. As an example, we'll show how to parse gene symbols, chromosome number and the corresponding SwissProt accession for the three genes.

In [1]:
# Start a Datasets gene API instance
api_client = ncbi.datasets.ApiClient()
ds_gene_instance = ncbi.datasets.GeneApi(api_client)

# Retrieve gene summaries for the three genes using NCBI Gene IDs
gene_summary = ds_gene_instance.gene_metadata_by_id([2798, 114814, 404718])

# Look up the symbols, chromosome number and SwissProt accession for each gene
def report_on_gene_descriptors(gene_summary, leader='\t', report_errors=True):
    if report_errors:
        for message in gene_summary.messages or []:
            print(f'{leader}Error for: ({",".join(message.error.invalid_identifiers)})')
            print(f'{leader}{leader}Reason: ({message.error.reason})')

    if not gene_summary.genes:
        print(f'{leader}No genes found')
        return

    for gene in map(lambda g: g.gene, gene_summary.genes):
        print(f'{leader}{gene.symbol} (GeneID: {gene.gene_id}), Chromosome: {gene.chromosomes}, SwissProt: {gene.swiss_prot_accessions}')

report_on_gene_descriptors(gene_summary)

	GNRHR2 (GeneID: 114814), Chromosome: ['1'], SwissProt: ['Q96P88']
	GNRHR (GeneID: 2798), Chromosome: ['4'], SwissProt: ['P30968']
	GNRHR2P1 (GeneID: 404718), Chromosome: ['14'], SwissProt: None


## Finding vertebrate gene orthologs by gene symbol

Now we're going to look for GnRHR genes in vertebrates. To find these genes, we'll first query the gene summary service by gene symbol and NCBI Taxonomy ID and retrieve the NCBI GeneID from the summary.

Next we'll query the gene ortholog service by NCBI GeneID to retrieve all vertebrate orthologs for the requested gene. Ortholog sets are calculated by NCBI Gene described here https://www.ncbi.nlm.nih.gov/kis/info/how-are-orthologs-calculated/.

Next we'll present a summary table for all orthologs including gene symbol, NCBI Gene ID, Chromosome, and SwissProt accession (if available).


In [1]:
# Get gene id by gene symbol + organism name
gene_symbol = 'GNRHR'
gene_taxon = 'human'
gene_descriptor = ds_gene_instance.gene_metadata_by_tax_and_symbol(symbols=[gene_symbol], taxon=gene_taxon)

if not gene_descriptor.genes:
    print(f'No gene found for {gene_taxon} {gene_symbol}')
else:
    gene_id = int(gene_descriptor.genes[0].gene.gene_id)

    # Query the gene ortholog service to get all vertebrate orthologs
    ortholog_set = ds_gene_instance.gene_orthologs_by_id(gene_id=gene_id)

    if not ortholog_set.ortholog_set_id:
        print(f'\nUnable to find orthologs for gene {gene_id}')
    else:
        orthologs_descriptors = ortholog_set.genes
        report_on_gene_descriptors(orthologs_descriptors, report_errors=False)


	GNRHR (GeneID: 100009509), Chromosome: ['15'], SwissProt: None
	GNRHR (GeneID: 100011217), Chromosome: ['5'], SwissProt: None
	GNRHR (GeneID: 100033874), Chromosome: ['3'], SwissProt: ['O18821']
	GNRHR (GeneID: 100093333), Chromosome: ['10'], SwissProt: None
	Gnrhr (GeneID: 100135532), Chromosome: ['Un'], SwissProt: ['Q8CH60']
	GNRHR (GeneID: 100385305), Chromosome: ['3'], SwissProt: None
	GNRHR (GeneID: 100437372), Chromosome: ['4'], SwissProt: None
	GNRHR (GeneID: 100483711), Chromosome: ['11'], SwissProt: None
	gnrhr (GeneID: 100552021), Chromosome: ['Un'], SwissProt: None
	GNRHR (GeneID: 100589627), Chromosome: ['9'], SwissProt: None
	GNRHR (GeneID: 100662022), Chromosome: ['Un'], SwissProt: None
	Gnrhr (GeneID: 100758518), Chromosome: ['1'], SwissProt: None
	GNRHR (GeneID: 100860755), Chromosome: ['6'], SwissProt: None
	GNRHR (GeneID: 100916844), Chromosome: ['6'], SwissProt: None
	GNRHR (GeneID: 100947835), Chromosome: ['Un'], SwissProt: None
	GNRHR (GeneID: 100967998), Chromoso

## Build a table of key metadata for GnRHR genes across vertebrates

Let's expand the taxonomic scope even further, and look at a selection of vertebrates.

We'll use a pre-determined list of Gene IDs to get gene summaries for these genes and build an easily readable table with key information about these genes.

In [1]:
# extract fields of interest from descriptors class to build a table
cols = '''
common_name
taxonomic_name
symbol
type
chromosome
num_transcripts
ensembl_id
omim_id
uniprot_id
nomenclature_id
nomenclature_auth
genome_coordinates
'''
cols = cols.split('\n')[1:-1]

def _range_repr(range):
    ret = []
    for interval in range:
        ret.append(f'{interval.begin}_{interval.end}')
    return ','.join(ret)

def _ranges_repr(ranges):
    ret = []
    for range in ranges:
        ret.append(f'{range.accession_version}:{_range_repr(range.range)}')
    return ','.join(ret)

# specify genes of interest and retrieve descriptors
gene_ids = [2798, 114814, 404718, 14715, 109324103, 109309182, 281798, 395368, 403718, 427517, 471226, 7226731, 100001586, 100135415, 100135416, 100135417, 100136028, 100270671, 100270672, 101318246, 101932446, 101935915, 101953943, 102193667, 102202954, 102205592, 102346610, 102363373, 102364206, 102366752, 102536567, 102687824, 102694185, 102770612, 103899900, 103899926, 105916404, 105919697, 105934126, 108392639, 109987527, 109994050, 109999298, 110488224, 110495632, 110496352, 110513414, 110520912, 112994411, 112996301, 114645297, 114667483]
gene_metadata = ds_gene_instance.gene_metadata_by_id(gene_ids)

# collect elements of the descriptor class into a dictionary based on each gene ID
table_data = {}
for g in gene_metadata.genes:
    if not g.gene:
        print(f'Gene not found: {g}')
        continue
    gene = g.gene

    table_data[gene.gene_id] = [gene.common_name]
    table_data[gene.gene_id].append(gene.taxname)
    table_data[gene.gene_id].append(gene.symbol)
    table_data[gene.gene_id].append(gene.type)
    table_data[gene.gene_id].append(gene.chromosome)
    if gene.transcripts:
        table_data[gene.gene_id].append(len(gene.transcripts))
    else:
        table_data[gene.gene_id].append(0)
    table_data[gene.gene_id].append(gene.ensembl_gene_ids)
    table_data[gene.gene_id].append(gene.omim_ids)
    table_data[gene.gene_id].append(gene.swiss_prot_accessions)
    if gene.nomenclature_authority:
        table_data[gene.gene_id].append(gene.nomenclature_authority.identifier)
        table_data[gene.gene_id].append(gene.nomenclature_authority.authority)
    else:
        table_data[gene.gene_id].append(None)
        table_data[gene.gene_id].append(None)        
    table_data[gene.gene_id].append(_ranges_repr(gene.genomic_ranges))

        
df = pd.DataFrame.from_dict(table_data, orient='index', columns=cols)
df.index.name = 'gene_id'
df

Gene not found: {'query': ['7226731'],
 'warnings': [{'gene_warning_code': 'DISCONTINUED_GENE_ID',
               'message': 'The gene you requested, (7226731) is a valid NCBI '
                          'Gene IDs that has been discontinued. It will be '
                          'omitted from your dataset. For more information '
                          'about the discontinued genes, visit NCBI Gene.\n',
               'reason': 'This GeneID has been discontinued.',
               'unrecognized_identifier': '7226731'}]}


,common_name,taxonomic_name,symbol,type,chromosome,num_transcripts,ensembl_id,omim_id,uniprot_id,nomenclature_id,nomenclature_auth,genome_coordinates
gene_id,,,,,,,,,,,,
100001586,zebrafish,Danio rerio,gnrhr4,PROTEIN_CODING,None,2,[ENSDARG00000038116],None,None,ZDB-GENE-050419-76,ZFIN,"NC_007129.7:25390740_25402909,NW_018395028.1:1..."
100135415,tropical clawed frog,Xenopus tropicalis,gnrhr2,PROTEIN_CODING,None,3,[ENSXETG00000021161],None,None,XB-GENE-5867415,Xenbase,NC_030679.2:109001475_109014876
100135416,tropical clawed frog,Xenopus tropicalis,gnrhr2/nmi,PROTEIN_CODING,None,2,[ENSXETG00000005637],None,None,None,None,NC_030679.2:116455054_116472596
100135417,tropical clawed frog,Xenopus tropicalis,gnrhr,PROTEIN_CODING,None,2,[ENSXETG00000001290],None,None,XB-GENE-5753573,Xenbase,NC_030684.2:142203708_142211556
100136028,rainbow trout,Oncorhynchus mykiss,gnrh-r,PROTEIN_CODING,None,1,[ENSOMYG00000000839],None,None,None,None,NC_048566.1:29925736_29930354
100270671,zebrafish,Danio rerio,gnrhr2,PROTEIN_CODING,None,1,[ENSDARG00000003553],None,None,ZDB-GENE-090128-3,ZFIN,"NC_007118.7:52364748_52368843,NW_018395268.1:1..."
100270672,zebrafish,Danio rerio,gnrhr1,PROTEIN_CODING,None,2,[ENSDARG00000100593],None,None,ZDB-GENE-090128-2,ZFIN,NC_007130.7:43213293_43235554
101318246,common bottlenose dolphin,Tursiops truncatus,GNRHR,PROTEIN_CODING,None,1,None,None,None,None,None,NC_047038.1:85406171_85428905
101932446,Painted turtle,Chrysemys picta,LOC101932446,PROTEIN_CODING,None,1,[ENSCPBG00000024942],None,None,None,None,NW_007281386.1:1139258_1141674


## Build a table showing GnRHR gene copy number across vertebrates

We're going to build a table showing how gene count varies in among a selected group of vertebrates.  
Note that rainbow trout has the most gene copies at 6, while numerous mammals, including mouse, dolphin and alpaca, only have a single annotated gene copy. 

In [1]:
# plot gene count based on organism
gene_cnt = df.groupby('common_name')['symbol'].count().reset_index()
gene_cnt.columns = ['organism', 'gene_count']
gene_cnt.sort_values('gene_count', ascending=False, inplace=True)
gene_cnt

,organism,gene_count
16,rainbow trout,6
8,coelacanth,4
20,zebrafish,3
2,Painted turtle,3
19,tropical clawed frog,3
4,ballan wrasse,3
15,mummichog,3
14,human,3
12,emu,2
18,spotted gar,2


## Use gene datasets to build a transcript-focused table

Finally, we are going to download a gene dataset for the human GnRHR genes, and use metadata included as part of the dataset to build a transcript-focused table.  
We'll take this metadata from the data report file, `data_report.jsonl`.

In [1]:
%%time

gene_ids = [2798, 114814, 404718]
gene_ds_download = ds_gene_instance.download_gene_package(gene_ids, _preload_content=False)

## write to a zip file 
zipfile_name = 'gene_ds.zip'
with open(zipfile_name, 'wb') as f:
    f.write(gene_ds_download.data)

print(f'Download saved to {zipfile_name}')

Download saved to gene_ds.zip
CPU times: user 3.19 ms, sys: 154 µs, total: 3.35 ms
Wall time: 74.4 ms


In [1]:
!unzip -v {zipfile_name}

Archive:  gene_ds.zip
 Length   Method    Size  Cmpr    Date    Time   CRC-32   Name
--------  ------  ------- ---- ---------- ----- --------  ----
     661  Defl:N      384  42% 07-16-2021 15:01 bc3c97af  README.md
    7081  Defl:N     1448  80% 07-16-2021 15:01 f96d97c3  ncbi_dataset/data/data_report.jsonl
    1434  Defl:N      478  67% 07-16-2021 15:01 fa82a51a  ncbi_dataset/data/data_table.tsv
     220  Defl:N      115  48% 07-16-2021 15:01 ddb3349b  ncbi_dataset/data/dataset_catalog.json
--------          -------  ---                            -------
    9396             2425  74%                            4 files


In [1]:
import zipfile
from google.protobuf.json_format import ParseDict
import jsonlines
import ncbi.datasets.v1.reports.gene_pb2 as gene_report_pb2
def gene_report_for(path_to_zipfile):
    '''
    Return an object representing the data report.
    path_to_zipfile: The relative path to the zipfile containing the virus data report
    '''
    gene_report = gene_report_pb2.GeneDescriptors()
    with zipfile.ZipFile(path_to_zipfile, 'r') as zip:      
        with zip.open('ncbi_dataset/data/data_report.jsonl') as file:
            reader = jsonlines.Reader(file)
            for gene_dict in reader.iter(type=dict, skip_invalid=True):
                ParseDict(gene_dict, gene_report.genes.add())
    return gene_report

def _5prime_len(transcript):
    if not transcript.cds or not transcript.cds.range:
        return None
    return transcript.cds.range[0].begin - 1

def _3prime_len(transcript):
    if not transcript.cds or not transcript.cds.range:
        return None
    return transcript.length - transcript.cds.range[0].end

gene_report = gene_report_for(zipfile_name)

rows = []
for gene in gene_report.genes:

    # transcripts for each gene are embedded as lists and require additional handling
    for transcript in gene.transcripts:
        rows.append({
            'gene_id': gene.gene_id,
            'gene_symbol': gene.symbol,
            'gene_taxonomy': gene.taxname,            
            'accVer': transcript.accession_version,
            'name': transcript.name,
            'length': transcript.length,
            '5`UTR_len': _5prime_len(transcript),
            '3`UTR_len': _3prime_len(transcript),
            'protAccVer': transcript.protein.accession_version or None,
            'protName': transcript.protein.isoform_name or None,
            'protLength': transcript.protein.length or None,
            'exonAccVer': transcript.exons.accession_version,
            'numExons': len(transcript.exons.range),
        })

transcript_table = pd.DataFrame(rows)

transcript_table


,gene_id,gene_symbol,gene_taxonomy,accVer,name,length,5`UTR_len,3`UTR_len,protAccVer,protName,protLength,exonAccVer,numExons
0,114814,GNRHR2,Homo sapiens,NR_002328.4,transcript variant 1,1626,NaN,NaN,None,None,NaN,NC_000001.11,3
1,114814,GNRHR2,Homo sapiens,NR_104034.1,transcript variant 3,786,NaN,NaN,None,None,NaN,NC_000001.11,3
2,114814,GNRHR2,Homo sapiens,NR_104033.1,transcript variant 2,1035,NaN,NaN,None,None,NaN,NC_000001.11,4
3,2798,GNRHR,Homo sapiens,NM_000406.3,transcript variant 1,4402,53.0,3362.0,NP_000397.1,isoform 1,328.0,NC_000004.12,3
4,2798,GNRHR,Homo sapiens,NM_001012763.2,transcript variant 2,4017,53.0,3214.0,NP_001012781.1,isoform 2,249.0,NC_000004.12,3
